<a href="https://colab.research.google.com/github/Ka1ux/Transfer_learning/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 1. Montar o Google Drive (com remontagem forçada para garantir)
drive.mount('/content/drive', force_remount=True)

# 2. Explorar a estrutura do Google Drive para encontrar o caminho correto
print("\nConteúdo do diretório raiz do Drive:")
try:
    print(os.listdir('/content/drive/MyDrive/'))
except FileNotFoundError:
    print("Could not list contents of /content/drive/MyDrive/. Ensure Google Drive is mounted correctly.")

# 3. Definir possíveis caminhos alternativos
possible_paths = [
    '/content/drive/MyDrive/minhas_imagens',
    '/content/drive/MyDrive/Colab Notebooks/minhas_imagens',
    '/content/drive/MyDrive/Datasets/minhas_imagens',
    '/content/drive/MyDrive/Imagens/minhas_imagens'
]

# 4. Encontrar o caminho correto automaticamente
base_dir = None
for path in possible_paths:
    if os.path.exists(path):
        base_dir = path
        print(f"\nDiretório encontrado em: {path}")
        break

if base_dir is None:
    print("\nERRO: Não foi possível encontrar o diretório 'minhas_imagens'")
    print("Sugestões:")
    print("1. Verifique se o nome da pasta está correto")
    print("2. Confira se as imagens estão em '/content/drive/MyDrive/'")
    print("3. A pasta deve conter subfolders for each class (e.g., 'cats', 'dogs')")
    print("\nConteúdo completo do seu Drive:")
    try:
        print(os.listdir('/content/drive/MyDrive/'))
    except FileNotFoundError:
        print("Could not list contents of /content/drive/MyDrive/. Ensure Google Drive is mounted correctly.")
    raise FileNotFoundError("Diretório 'minhas_imagens' não encontrado")

# 5. Verificar a estrutura das pastas
print("\nEstrutura encontrada:")
try:
    for item in os.listdir(base_dir):
        item_path = os.path.join(base_dir, item)
        if os.path.isdir(item_path):
            try:
                num_images = len(os.listdir(item_path))
                print(f"{item}: {num_images} imagens")
            except OSError as e:
                print(f"Error listing images in {item}: {e}")
except OSError as e:
    print(f"Error listing contents of directory {base_dir}: {e}")


# 6. Parâmetros do modelo
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 10

# 7. Pré-processamento com aumento de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator( # Use a separate generator for validation without augmentation
    rescale=1./255,
    validation_split=0.2
)

# 8. Carregar imagens usando os geradores
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Assuming binary classification
    subset='training',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory( # Use val_datagen here
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Assuming binary classification
    subset='validation',
    shuffle=False
)

# Verificar as classes encontradas
print("\nClasses encontradas:", train_generator.class_indices)


# 9. Criar modelo Xception (ou outro modelo pré-treinado)
base_model = tf.keras.applications.Xception(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False # Congelar as camadas convolucionais

# 10. Adicionar camadas de classificação ao modelo base
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') # Camada de saída para classificação binária
])

# 11. Compilar o modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy', # Função de perda para classificação binária
    metrics=['accuracy']
)

# 12. Treinar o modelo
print("\nIniciando o treinamento do modelo...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# 13. Avaliar o modelo
print("\nAvaliando o modelo...")
loss, accuracy = model.evaluate(validation_generator)
print(f'\nAcurácia no conjunto de validação: {accuracy * 100:.2f}%')